In [18]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!unzip '/content/drive/My Drive/flower_data/102flowers.zip'

unzip:  cannot find or open /content/drive/My Drive/flower_data/102flowers.zip, /content/drive/My Drive/flower_data/102flowers.zip.zip or /content/drive/My Drive/flower_data/102flowers.zip.ZIP.


In [20]:
!pip install tensorflow_addons

In [21]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.layers import InputLayer, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


**Read json file**

In [22]:
import json

# Replace 'your_file_name.json' with the path to your uploaded JSON file
file_path = 'flower_data/cat_to_name.json'

# Open and read the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Now 'data' is a Python dictionary containing the contents of the JSON file
print(data)


FileNotFoundError: [Errno 2] No such file or directory: 'flower_data/cat_to_name.json'

**Data augmentation and preprocessing**

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/flower_data/train'
test_dir = '/content/flower_data/valid'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)  # Use 20% of training data for validation

validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')


In [ ]:
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

train_dir = '/content/flower_data/train'

class_names = os.listdir(train_dir)
class_names.sort()

plt.figure(figsize=(10, 10))

for i, class_name in enumerate(class_names):

    class_files = os.listdir(os.path.join(train_dir, class_name))

    if class_files:
        first_image_file = class_files[0]

        img_path = os.path.join(train_dir, class_name, first_image_file)

        img = image.load_img(img_path, target_size=(224, 224))

        plt.subplot(len(class_names)//10 + 1, 10, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(img)
        plt.xlabel(class_name)
    else:
        print(f"No images found in {class_name}.")

#Show all classes
plt.tight_layout()
plt.show()


**Model - EfficientNet**

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Custom layers on top for our specific task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(102, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


**Train frozen based model**

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)


**Train unfroen based model**

In [ ]:
# Unfreeze the base model
base_model.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)


**Plot accuracy and loss**

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history_fine.history["accuracy"])
plt.plot(history_fine.history['val_accuracy'])
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

**Evaluate model on test dataset**

In [ ]:
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)

print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Train on additional 5 epoch

In [ ]:
# Unfreeze the base model
base_model.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)


In [ ]:
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)

print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

**Save the model**

In [ ]:
from keras.models import load_model

model.save('flowers_efficientnet_model.h5')